<h1>Парсинг данных</h1>

Задача: Оценка рыночной стоимости мотоцикла с пробегом 

Сайт с данными: https://moto.drom.ru/ 

Целевая переменная: цена в рублях

Признаки: 
1. Модель мотоцикла
2. Пробег
3. Класс 
4. Год выпуска 
5. Объем двигателя 
6. Число тактов 
7. Состояние 
8. Документы
9. Город 
10. Дата публикации объявления 

Как они будут называться в таблице, разделитель ',': 

target: price

1. model
2. mileage 
3. motorcycle_class
4. year 
5. engine_capacity
6. engine_strokes 
7. damaged
8. documents
9. city
10. date 

Алгоритм:
1. Скачать все страницы (есть номер страницы, если забанят - знаю, откуда продолжить)
2. Получить из них список ссылкок (просто парсинг)
3. Потом идти по каждой ссылке (есть номер ссылки, если забанят - знаю, откуда продолжить)

<b>Импорт библиотек</b>

In [1]:
import numpy as np
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup

<b>Класс, описывающий мотоцикл</b>

In [2]:
class Motorcycle:
    def __init__(self, soup_motorcycle, index=None):
        self.soup = soup_motorcycle
        if (index is None):
            self.index = 0
        else:
            self.index = index
            
        self.warning_string_ = "Index: " + str(self.index) + "; Warning: "
        
    def warning_(self, string):
        print(self.warning_string_ + string)
        
    def parse_information(self):
        self.price = self.get_price()
        if (self.price is None):
            self.warning_('price not found')
            return False
        
        self.model = self.get_feature({"data-field" : "model"})
        self.mileage = self.get_feature({"data-field" : "motoMileage"})
        self.motorcycle_class = self.get_feature({"data-field" : "motoBodyType"})
        self.year = self.get_feature({"data-field" : "year"})
        self.engine_capacity = self.get_feature({"data-field" : "displacement"})
        self.engine_strokes = self.get_feature({"data-field" : "motoEngineType"})
        self.damaged = self.get_feature({"data-field" : "motoDriveCondition"})
        self.documents = self.get_feature({"data-field" : "hasDocuments"})
        self.city = self.get_feature({"title" : "Выбранный город"})
        
        self.date = self.get_feature({"class" : "viewbull-header__actuality"})
        comma_index = self.date.find(',')
        if (comma_index != -1):
            self.date = self.date[comma_index+2:]
            
        return True
        
    def get_price(self):
        price = self.soup.select_one('span.viewbull-summary-price__value')
        if (price is not None):
            price = price.text.strip()
        else:
            price = None
        return price
    
    def get_feature(self, attrs):
        feature = self.soup.findAll(attrs=attrs)
        if (len(feature) > 0):
            feature = feature[0].text.strip()
        else:
            feature = None
        return feature

    def __repr__(self):
         return "Motorcycle class"
        
    def __str__(self):
        string = str(self.index) + ',' + str(self.price) + ',' + str(self.model) + ',' + \
                    str(self.mileage) + ',' + str(self.motorcycle_class) + ',' + \
                    str(self.year) + ',' + str(self.engine_capacity) + ',' + \
                    str(self.engine_strokes) + ',' + str(self.damaged) + ',' + \
                    str(self.documents) + ',' + str(self.city) + ',' + str(self.date)
        return string

<b>Информация по подключению</b>

In [3]:
url_template = 'https://moto.drom.ru'
url_list = url_template + '/moskva/sale/?status=archive&page='

In [4]:
# Сайт с прокси: http://spys.one/proxys/RU/

headers = None
proxies = None
#proxies = { 'https': '5.53.19.82:56907' }

<b>Вспомогательные функции для подключения</b>

In [5]:
def get_links(url_page, proxies=None, headers=None, sleep=False, sleep_seconds=5):
    html_page = requests.get(url_page, proxies=proxies, headers=headers).content
    soup = BeautifulSoup(html_page, 'html.parser')

    if (sleep):
        time.sleep(np.random.randint(sleep_seconds))
          
    return soup.select('.bulletinLink')

In [6]:
def get_motorcycle_soup(url_motorcycle, proxies=None, headers=None, sleep=False, sleep_seconds=5):
    html_motorcycle = requests.get(url_motorcycle, proxies=proxies, headers=headers).content
    soup_motorcycle = BeautifulSoup(html_motorcycle, 'html.parser')

    if (sleep):
        time.sleep(np.random.randint(sleep_seconds))
        
    return soup_motorcycle

<b>Создание списка ссылок</b>

In [7]:
current_page = 1
last_page = 142
current_link = 1

with open('data/links.txt', 'a') as f_output:
    for page in range(current_page, last_page+1):
        links = get_links(url_list + str(page), proxies, headers)

        if (links is None):
            print("Error while getting motorcycle links")
            print("Page: " + str(page))
            print("Link: " + str(current_link))
            break

        for link in links:
            print(str(current_link) + '. ' + url_template + link['href'], file=f_output)
            current_link += 1

<b>Парсинг страниц с мотоциклами</b>

In [9]:
features = "id,price,model,mileage,motorcycle_class,year,engine_capacity,engine_strokes,damaged,documents,city,date"

with open('data/links.txt', 'r') as links_file:
    with open('data/motorcycles.csv', 'a', encoding='utf-8') as motorcycle_file:
        print(features, file=motorcycle_file)
        for i, link in enumerate(links_file, 1):
            link = link[link.find('h'):]
            
            soup = get_motorcycle_soup(link, proxies, headers, False)
            if (soup is None):
                print("Error while getting motorcycle information")
                print("Link: " + str(link))
                break

            motorcycle = Motorcycle(soup, i)
            if (motorcycle.parse_information()):
                print(motorcycle, file=motorcycle_file)
            else:
                print("Error while parsing motorcycle information")
                print("Link: " + str(link))

Index: 94; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodam-honda-cbr929-66695315.html

Index: 100; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodaju-ktm-690-duke-2012-g-v-66639732.html

Index: 165; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/honda-cbr600f-66032691.html

Index: 241; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/prodam-motocikl-ural-s-koljaskoj-8.103-10-65426129.html

Index: 376; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/elektricheskij-motocikl-2000-3000w-kawasaki-devil-64336390.html

Index: 397; Warning: price not found
Error while parsing motorcycle information
Link: https://moto.drom.ru/moskva/sale/motocikl-minsk-d4-125-63260830.html

Index: 848; Warning